# Implementing a Sample Trading Strategy

In [ ]:
pip install yfinance

In [2]:
import yfinance as yf 

# Example: Fetch 1 year of daily price for TCS
ticker = yf.Ticker("TCS.NS")
data = ticker.history(period="1y")


In [3]:
print(data.head())

                                  Open         High          Low        Close  \
Date                                                                            
2024-07-16 00:00:00+05:30  4091.950422  4114.067556  4065.370303  4098.276855   
2024-07-18 00:00:00+05:30  4080.180398  4242.014476  4080.180398  4232.745605   
2024-07-19 00:00:00+05:30  4244.395285  4285.046177  4212.789018  4229.648926   
2024-07-22 00:00:00+05:30  4227.241043  4246.902857  4192.881831  4214.854004   
2024-07-23 00:00:00+05:30  4210.577198  4287.946623  4194.159392  4229.600098   

                            Volume  Dividends  Stock Splits  
Date                                                         
2024-07-16 00:00:00+05:30  2029521        0.0           0.0  
2024-07-18 00:00:00+05:30  5613364        0.0           0.0  
2024-07-19 00:00:00+05:30  3903354       10.0           0.0  
2024-07-22 00:00:00+05:30  1896386        0.0           0.0  
2024-07-23 00:00:00+05:30  2365208        0.0           0.0 

# Backtesting Setup 

## Strategy:

If insider buys ₹10L+ of stock on trade_date,
we buy the stock next trading day, hold it for 30 days,
then sell and calculate return.

In [4]:
# CODE ON ONE ROW 

import pandas as pd
import yfinance as yf 
from datetime import timedelta

In [ ]:
df = pd.read_csv('nse_insider_cleaned.csv')

# filter only large BUY transactions 
df = df[(df['type'] == 'Buy') & (df['value'] > 10_00_000)]

# Convert trade_date to datetime 
df['trade_date'] = pd.to_datetime(df['trade_date'], errors='coerce')
df = df.dropna(subset = ['trade_date']); 


In [9]:
# convert the first trade for testing 
row = df.iloc[0]; 
symbol = row['symbol'] + ".NS" 
entry_date= row['trade_date'] + timedelta(days=1); # buy the next day 
exit_date = row['trade_date'] + timedelta(days=30); # sell after 30 days 

# download price data from yahoo finance 
stock = yf.Ticker(symbol); 
price_data = stock.history(start=entry_date - timedelta(days=5), end=exit_date + timedelta(days=5)); 

# After downloading price data
price_data = stock.history(start=entry_date - timedelta(days=5), end=exit_date + timedelta(days=5))

# Remove timezone from index to fix comparison
price_data.index = price_data.index.tz_localize(None)

# Now comparisons will work
entry_price_row = price_data.loc[price_data.index >= entry_date].head(1)
exit_price_row = price_data.loc[price_data.index >= exit_date].head(1)


if not entry_price_row.empty and not exit_price_row.empty: 
    entry_price = entry_price_row['Close'].values[0]; 
    exit_price = exit_price_row['Close'].values[0]; 
    
    return_pct = ((exit_price - entry_price)/entry_price) * 100; 
    print(f"📈 Insider trade on: {row['trade_date'].date()}")
    print(f"🟢 Bought {symbol} on {entry_price_row.index[0].date()} at ₹{entry_price:.2f}")
    print(f"🔴 Sold {symbol} on {exit_price_row.index[0].date()} at ₹{exit_price:.2f}")
    print(f"💰 Return: {return_pct:.2f}%")
else:
    print("❌ Price data not available for entry or exit date")

📈 Insider trade on: 2024-12-31
🟢 Bought ALMONDZ.NS on 2025-01-01 at ₹33.50
🔴 Sold ALMONDZ.NS on 2025-01-30 at ₹29.96
💰 Return: -10.57%
